In [ ]:
import dlt
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import from_json, col, avg, count, window, expr, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from functools import reduce

In [ ]:
# In production use a secret for the bootstrap_servers, this is just for demo purposes and it is deleted after each demo
KAFKA_BOOTSTRAP_SERVERS = "b-1.msksilveraiwolfuseast.nqmaxv.c11.kafka.us-east-1.amazonaws.com:9092,b-2.msksilveraiwolfuseast.nqmaxv.c11.kafka.us-east-1.amazonaws.com:9092"

# Multiplex Bronze

A Multiplex Bronze Streaming Table is a data table in a lakehouse architecture designed to ingest and store raw streaming data from multiple sources into a single, unified table, rather than maintaining separate bronze tables for each topic. This approach is commonly used in Databricks and Delta Lake architectures, particularly within the medallion architecture, which organizes data into three layers:

- Bronze Layer (Raw Data) – Stores unprocessed, raw data from various sources, including real-time streams.
- Silver Layer (Cleansed Data) – Data is cleaned, deduplicated, and formatted for structured analytics.
- Gold Layer (Aggregated Data) – Optimized for business intelligence and reporting.

**Breaking Down "Multiplex Bronze Streaming Table"**
- Multiplex: Handles multiple streaming topics (e.g., Kafka topics, event hubs) within the same table.
- Bronze: Represents the raw ingestion layer where all data lands before transformation.
- Streaming Table: Supports continuous data ingestion from real-time sources.

A Multiplex Bronze Table is beneficial in scenarios where multiple event streams or sources are ingested at scale. Here’s why:

✅ Simplified Data Ingestion

Instead of managing separate ingestion pipelines for each streaming topic, a single pipeline ingests all raw data into one table.
Reduces operational overhead in pipeline development and maintenance.


✅ Better Query Performance for Downstream Consumers

When Silver and Gold tables need to process data from multiple sources, a single source of truth (multiplex table) simplifies joins and aggregations.
Avoids the need for complex union queries across multiple bronze tables.

✅ Improved Governance and Observability

Centralized auditing and logging for all ingested data.
Easier to apply access control policies, monitoring, and data quality checks in one place.

✅ Faster Processing & Streaming Performance

With Delta Lake's metadata handling and compaction features, a single partitioned table (e.g., partitioned by event_type) provides faster lookups than multiple small tables.
Reduces the risk of small file problems that arise with many separate topic-based tables.


In [ ]:
def read_kafka_stream(topic: str):
    return (
        spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)
        .option("subscribe", topic)
        .option("startingOffsets", "earliest")
        .load()
        .selectExpr("CAST(value AS STRING) AS parsed_value", "CAST(key AS STRING) AS parsed_key", "*")
    )

@dlt.table(
    name="multipex_bronze",
    comment="Bronze table for streaming ingestion",
    table_properties={
        "pipelines.reset.allowed": "false",
        "quality": "bronze"
    }
)
def multipex_bronze():
    topics = [
        "orders-raw", 
        "shipment-contents", 
        "inventory-updates", 
        "shipment-status", 
        "customer-feedback"
    ]
    
    # Read all topics and union them into a single dataframe
    streams = [read_kafka_stream(topic).withColumn("topic", expr(f"'{topic}'")) for topic in topics]
    return reduce(lambda df1, df2: df1.union(df2), streams)


# Silver

A Silver Table in a data lakehouse architecture refers to a cleansed and enriched dataset that sits between the raw Bronze layer and the curated Gold layer. It is part of the medallion architecture used in Databricks, Delta Lake, and modern data platforms.

Medallion Architecture Overview
- Bronze Tables (Raw Data) – Stores raw, unprocessed data from various sources.
- Silver Tables (Cleansed Data) – Processes, cleans, and deduplicates data from Bronze before making it available for analytics.
- Gold Tables (Aggregated Data) – Optimized for business intelligence (BI), reporting, and machine learning.

**What Makes a Silver Table Different?**

✅ Cleansed & Preprocessed Data

Removes duplicates, corrupt records, and null values.
Standardizes formats (e.g., date/time, data types).

✅ Joins & Enrichments

Combines data from multiple Bronze tables.
Adds reference data, metadata, or calculated columns.

✅ Optimized for Query Performance

Uses Delta Lake optimizations (Z-ordering, indexing, caching).
Reduces unnecessary scans by partitioning data.

✅ Supports Incremental Loads

Uses Change Data Capture (CDC) or merge operations to update records efficiently.
Handles slowly changing dimensions (SCDs) in a structured way.

✅ Prepares Data for Gold Layer

Feeds aggregated, business-ready datasets into Gold tables for analytics and BI tools.

**Why Use Silver Tables?**
- Improves Data Quality – Ensures clean, trustworthy data before analytics.
- Enhances Performance – Queries run faster compared to raw Bronze data.
- Reduces Complexity in Gold Tables – Keeps business-layer datasets optimized and structured.
- Supports Machine Learning & AI – ML models require structured, well-processed data.

## Table Schemas & Rules

In [ ]:
# Define schema structures
orders_schema = StructType([
    StructField("order_id", StringType()),
    StructField("product_id", StringType()),
    StructField("quantity", IntegerType()),
    StructField("event_timestamp", TimestampType())
])

shipment_content_schema = StructType([
    StructField("shipment_id", StringType()),
    StructField("order_id", StringType()),
    StructField("product_id", StringType()),
    StructField("quantity", IntegerType()),
    StructField("event_timestamp", TimestampType())
])

inventory_schema = StructType([
    StructField("warehouse_location", StringType()),
    StructField("product_id", StringType()),
    StructField("stock_level", IntegerType()),
    StructField("event_timestamp", TimestampType())
])

shipment_status_schema = StructType([
    StructField("shipment_id", StringType()),
    StructField("order_id", StringType()),
    StructField("current_status", StringType()),
    StructField("event_timestamp", TimestampType())
])

customer_feedback_schema = StructType([
    StructField("order_id", StringType()),
    StructField("customer_id", StringType()),
    StructField("rating", IntegerType()),
    StructField("comment", StringType()),
    StructField("event_timestamp", TimestampType())
])

In [ ]:
# Define streaming tables
def parse_stream(schema, topic):
    return (
        dlt.readStream("multipex_bronze")
        .filter(col("topic") == topic)
        .select(from_json(col("parsed_value"), schema).alias("json_value"), "parsed_key")
        .selectExpr("parsed_key AS key", "json_value.*")
    )

def parse_read(schema, topic):
    return (
        spark.read.table("multipex_bronze")
        .filter(f"topic = '{topic}'")
        .select(from_json(col("parsed_value"), schema).alias("json_value"), "parsed_key")
        .selectExpr("parsed_key AS key", "json_value.*")
    )

order_rules = {
    "invalid_qty": "quantity != 0"
}

quarentine_rules = {}
quarentine_rules["invalid_records"] = f"NOT({' AND '.join(order_rules.values())})"

## Table Definition and Data Constraints

In [ ]:
@dlt.table(
    name="orders_silver",
    comment="Aggregated orders partition"
)
@dlt.expect_all_or_drop(order_rules)
def orders_silver():
    return parse_stream(orders_schema, "orders-raw")


@dlt.table(
    name="orders_quarentine_silver",
    comment="Aggregated orders partition that have been cancelled"
)
@dlt.expect_all_or_drop(quarentine_rules)
def orders_quarentine_silver():
    return parse_stream(orders_schema, "orders-raw")

### Change Data Capture
---

Change Data Capture (CDC) is a technique used to track and capture changes (INSERTS, UPDATES, and DELETES) in a source dataset and apply them efficiently to a target dataset. In the context of Delta Live Tables (DLT), CDC helps in managing incremental data updates in a declarative, scalable, and automated way.

DLT is a framework built on Apache Spark and Delta Lake that simplifies building ETL (Extract, Transform, Load) pipelines. It provides built-in support for CDC by enabling you to capture changes from streaming or batch sources and apply them efficiently to your target Delta table.

See the official documentation here: https://docs.databricks.com/aws/en/dlt/cdc

#### Change Data Capture Materialized View Example

This materialized view mimics the behavior of CDC by deduplicating records using a window function and enforcing a constraint to keep only the latest record per shipment_id and product_id.
It allows you to preview the transformations that apply_changes() would apply before using it in a live CDC pipeline.

> This is not a best practice way of doing CDC but it shows the idea of deduplicating and applying the latest changes.

In [ ]:
@dlt.table(
    name="shipment_content_silver",
    comment="A materialized view to show what the apply_changes() will do"
)
@dlt.expect_all_or_drop({"duplicated_records": "row_num = 1"})
def shipment_content_silver():
    df = parse_read(shipment_content_schema, "shipment-contents")

    # Add a row number to the dataframe to identify the latest record
    window_spec = Window.partitionBy("shipment_id", "product_id").orderBy(col("event_timestamp").desc())
    df_with_row_num = df.withColumn("row_num", row_number().over(window_spec))

    return df_with_row_num


#### Delta Live Table CDC Slow Changing Dimension

Slowly Changing Dimensions (SCD) refer to a technique for managing historical changes in a dataset, particularly for dimensional tables in a data warehouse. In Delta Live Tables (DLT), SCDs can be managed using the APPLY CHANGES feature.

There are two commonly used SCD types in Delta Live Tables:

1. **SCD Type 1 (Overwrite)**
   - SCD Type 1 does not retain history.
   - When a change occurs (e.g., an update in the source data), the existing record is overwritten.
   - This approach ensures only the latest data is stored.
   - **Use Case:**
     - Used when historical tracking is not required.
     - Best for scenarios where corrections or updates must replace old data.
     - Example: Correcting customer contact details (phone number, email).

2. **SCD Type 2 (Historical)**
   - SCD Type 2 preserves historical records.
   - When a change occurs, a new row is inserted with a different start_date and end_date, while the previous record is marked as inactive.
   - This allows tracking historical changes over time.
   - **Use Case:**
     - Used when maintaining a history of changes is necessary.
     - Best for tracking customer address changes, product price changes, employee role history, etc.

In [ ]:
@dlt.view(
    name="shipment_content_source_silver",
    comment="Source table for the apply_changes() function"
)
def shipment_content_source_silver():
    return parse_stream(shipment_content_schema, "shipment-contents")

In [ ]:
dlt.create_streaming_table(
    name="shipment_content_scd1_silver",
    comment="Slowly changing dimensional table with updates",
)
dlt.apply_changes(
    target = "shipment_content_scd1_silver",
    source = "shipment_content_source_silver",
    keys = ["shipment_id", "product_id"],
    sequence_by = col("event_timestamp"),
    stored_as_scd_type = 1
)

In [ ]:
dlt.create_streaming_table(
    name="shipment_content_scd2_silver",
    comment="Slowly changing dimensional table without updates",
)
dlt.apply_changes(
    target = "shipment_content_scd2_silver",
    source = "shipment_content_source_silver",
    keys = ["shipment_id", "product_id"],
    sequence_by = col("event_timestamp"),
    stored_as_scd_type = 2
)

#### Applying SCD Type 1 to the Rest

In [ ]:
# Inventory 
@dlt.view(
    name="inventory_source_silver",
    comment="Source view for the apply_changes function"
)
def inventory_source_silver():
    return parse_stream(inventory_schema, "inventory-updates")

dlt.create_streaming_table(
    name="inventory_scd1_silver",
    comment="Slowly changing dimensional table with updates",
)
dlt.apply_changes(
    target = "inventory_scd1_silver",
    source = "inventory_source_silver",
    keys = ["warehouse_location", "product_id"],
    sequence_by = col("event_timestamp"),
    stored_as_scd_type = 1
)
    
# Shipment Status
@dlt.view(
    name="shipment_status_source_silver",
    comment="Source view for the apply_changes function"
)
def shipment_status_silver():
    return parse_stream(shipment_status_schema, "shipment-status")

dlt.create_streaming_table(
    name="shipment_status_scd1_silver",
    comment="Slowly changing dimensional table with updates",
)
dlt.apply_changes(
    target = "shipment_status_scd1_silver",
    source = "shipment_status_source_silver",
    keys = ["shipment_id", "order_id"],
    sequence_by = col("event_timestamp"),
    stored_as_scd_type = 1
)


# Customer Feedback
@dlt.view(
    name="customer_feedback_source_silver",
    comment="Source view for the apply_changes function"
)
def customer_feedback_silver():
    return parse_stream(customer_feedback_schema, "customer-feedback")

dlt.create_streaming_table(
    name="customer_feedback_scd1_silver",
    comment="Slowly changing dimensional table with updates",
)
dlt.apply_changes(
    target = "customer_feedback_scd1_silver",
    source = "customer_feedback_source_silver",
    keys = ["order_id", "customer_id"],
    sequence_by = col("event_timestamp"),
    stored_as_scd_type = 1
)

# Gold
---

Gold tables represent the final, refined, and business-ready layer of the data pipeline. They contain aggregated, enriched, and highly optimized data, ready for business intelligence (BI), analytics, and reporting.

**Purpose of Gold Tables**
1. Business-Ready Analytics
Gold tables store cleaned and transformed data tailored for business decision-making.
They provide aggregated KPIs, trends, and metrics that can be used directly by analysts and executives.

2. Performance Optimization for Reporting & Queries
Since Gold tables store pre-aggregated and denormalized data, they reduce the need for heavy computations in real-time queries.
This improves query performance for BI tools like Power BI, Tableau, or Looker.

3. Simplification of Data Access
Gold tables make it easy for non-technical users (e.g., business analysts) to access data without complex SQL queries.
Instead of navigating multiple Silver tables, users get one source of truth.

4. Reducing Compute Costs
Since complex transformations (joins, aggregations) are precomputed in Gold tables, the overall query cost is reduced.
This is critical in big data environments like Databricks where compute costs can be high.

5. Enabling Machine Learning & AI
Gold tables often serve as the feature store for machine learning models.
They provide cleaned, aggregated, and structured datasets that are ready for model training.

Gold tables bridge raw data with actionable insights, making data easily consumable for business, BI, and machine learning. They improve performance, simplify data access, and reduce costs, making them essential in modern data pipelines. 🚀

In [ ]:
@dlt.table(
    name="wide_orders_gold",
    comment="Final wide table combining all silver tables"
)
@dlt.expect_all_or_drop({"no_shipment": "shipment_id IS NOT NULL"})
def wide_orders_gold():
    orders = dlt.read("orders_silver")
    shipments = (dlt.read("shipment_content_scd1_silver")
        .drop("key", "event_timestamp")
        .withColumnRenamed("quantity", "shipped_quantity")
    )
    inventory = (dlt.read("inventory_scd1_silver")
        .drop("key", "event_timestamp")             
    )
    shipment_status = (dlt.read("shipment_status_scd1_silver")
        .drop("key", "event_timestamp")
    )
    feedback = (dlt.read("customer_feedback_scd1_silver")
        .drop("key", "event_timestamp")
    )
    
    return (
        orders
        .join(shipments, ["order_id", "product_id"], "left")
        .join(inventory, "product_id", "left")
        .join(shipment_status, ["order_id", "shipment_id"], "left")
        .join(feedback, "order_id", "left")
    )